# RAG System

In [ ]:
import tiktoken
from langchain_openai import OpenAIEmbeddings
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

open_api_key = ''

os.environ['OPENAI_API_KEY'] = open_api_key

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

Count tokens

In [3]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [4]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)


1536

In [5]:
print(query_result)

[-0.0035892894957214594, -0.0030708718113601208, -0.012454708106815815, -0.024972829967737198, -0.028308460488915443, 0.01755327545106411, -0.01554935984313488, -0.008535659871995449, -0.006747357081621885, -0.00487027270719409, -0.003703436581417918, 0.005456861574202776, -0.008351756259799004, -0.001444276887923479, 0.009429811500012875, 0.01955718919634819, 0.03076896257698536, 0.01491521019488573, 0.01570155657827854, -0.015473262406885624, -0.016107412055134773, 3.998118336312473e-05, 0.006664917338639498, -0.024389412254095078, -0.01263861171901226, -0.0032151408959180117, 0.011941046454012394, -0.0013158614747226238, -0.018124010413885117, -0.015815703198313713, 0.051188595592975616, -0.0031025793869048357, -0.022880135104060173, -0.01207421813160181, 0.0022005008067935705, 0.002760138362646103, 0.01438252441585064, 0.004844906739890575, 0.013608860783278942, 0.00035928565193898976, -0.0013491542777046561, 0.020952319726347923, 0.01562545821070671, -0.0045151486992836, 0.0140527

Cosine Similarity

In [6]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915835035412


Retrieval (documents in vector space) augmented generation (llm)

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://mathmod.chnu.edu.ua/",)
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
blog_docs = loader.load()

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits)

# splits = blog_docs.split(' ')
# print(splits)

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [9]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

: 